# 构建卷积神经网络

* 卷积网络中的输入和层与传统神经网路有点区别，需重新设计，训练模块基本一致

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# 首先读取数据

* 分别构建训练集和测试集（验证集）
* Dataloader 来迭代取数据

In [5]:
# 定义超参数
input_size = 28          # 图像的总尺寸 28 * 28
num_classes = 10         # 标签的种类数
num_epoches = 3          # 训练的总循环周期
batch_size = 64          # 一个批次的大小，64 张图片

In [6]:
# 数据划分
# 训练集
train_dataset = datasets.MNIST(
    root='../../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
# 测试集
test_dataset = datasets.MNIST(
    root='../../data',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

# 构建 batch 数据
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
)

# 卷积网络模块构建

* 一般卷积层、relu层、池化层可以写成一个套餐
* 注意卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(             # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,                  # 灰度图
                out_channels=16,                # 要得到几个特征图
                kernel_size=5,                  # 卷积核大小
                stride=1,                       # 步长
                padding=2,                      # 边缘填充
            ),                                  # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                          # relu 层
            nn.maxPool2d(kernel_size=2),        # 池化层 输出结果为 (16, 14, 14)
        )
        self.conv2 = nn.Sequential(             # 下一个套餐的输入
            nn.Conv2d(16, 32, 5, 1, 2),         # 输出  (32, 14, 14)
            nn.ReLU(),                          # relu 层
            nn.MaxPool2d(2),                    # 输出层 （32, 7, 7）
        )
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return(output)
